In [32]:
import pandas as pd

# to generate this, click on goodreads "my books".  Then click "print".  Then set the &per_page=1000 query param
books = pd.read_html("book_tables.txt")[0]

In [33]:
books

,cover,title,author,avg rating,rating,shelves,date read,date added
0,cover,"title The Forever War (The Forever War, #1)","author Haldeman, Joe",avg rating 4.14,my rating 1 of 5 stars2 of 5 stars3 of 5 star...,shelves read[edit],date read not set [edit],"date added Aug 26, 2012"
1,cover,title The Smartest Guys in the Room: The Amazi...,"author McLean, Bethany",avg rating 4.20,my rating 1 of 5 stars2 of 5 stars3 of 5 star...,shelves read[edit],date read not set [edit],"date added Jul 25, 2019"
2,cover,title Battle Cry of Freedom,"author McPherson, James M.",avg rating 4.36,my rating 1 of 5 stars2 of 5 stars3 of 5 star...,shelves read[edit],date read not set [edit],"date added Jul 25, 2019"
3,cover,title The Mask of Command,"author Keegan, John",avg rating 4.12,my rating 1 of 5 stars2 of 5 stars3 of 5 star...,shelves read[edit],date read not set [edit],"date added Jul 25, 2019"
4,cover,title Endurance,"author Worsley, Frank A.",avg rating 4.46,my rating 1 of 5 stars2 of 5 stars3 of 5 star...,shelves read[edit],date read not set [edit],"date added Oct 25, 2017"
...,...,...,...,...,...,...,...,...
654,cover,title The Trials of Koli (Rampart Trilogy #2),"author Carey, M.R. *",avg rating 4.28,my rating 1 of 5 stars2 of 5 stars3 of 5 star...,shelves to-read[edit],date read not set [edit],"date added Jul 12, 2021"
655,cover,title The Fall of Koli (Rampart Trilogy #3),"author Carey, M.R. *",avg rating 4.45,my rating 1 of 5 stars2 of 5 stars3 of 5 star...,shelves to-read[edit],date read not set [edit],"date added Jul 12, 2021"
656,cover,title Frank Herbert's Dune Saga Collection: Bo...,"author Herbert, Frank",avg rating 4.46,my rating 1 of 5 stars2 of 5 stars3 of 5 star...,shelves to-read[edit],date read not set [edit],"date added Jul 23, 2021"
657,cover,title Razorblade Tears,"author Cosby, S.A. *",avg rating 4.16,my rating 1 of 5 stars2 of 5 stars3 of 5 star...,shelves to-read[edit],date read not set [edit],"date added Jul 10, 2021"


In [34]:
books["title"] = books["title"].str.replace("title ", "")

In [35]:
books.loc[0]["rating"]

'my rating  1 of 5 stars2 of 5 stars3 of 5 stars4 of 5 stars[ 5 of 5 stars ]'

In [36]:
ratings_string = books["rating"].str.extract("(\[.+\])")

In [37]:
books["rating"] = ratings_string[0].str.slice(start=2, stop=3)

In [38]:
books = books[["title", "rating"]]

In [39]:
titles = pd.read_json("books_titles.json")
titles["book_id"] = titles["book_id"].astype(str)

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

tfidf = vectorizer.fit_transform(titles["mod_title"])

In [41]:
import re
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search(query,vectorizer):
    processed = re.sub("[^a-zA-Z0-9 ]", "", query.lower())
    query_vec = vectorizer.transform([query])
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indices = np.argpartition(similarity, -10)[-10:]
    results = titles.iloc[indices]
    results = results.sort_values("ratings", ascending=False)
    
    return results.iloc[0,:]["book_id"]

In [42]:
books["book_id"] = books["title"].apply(lambda x: search(x, vectorizer))

/var/folders/xz/9z84c__j28g8tg28bmcthjj00000gn/T/ipykernel_34151/4009849999.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books["book_id"] = books["title"].apply(lambda x: search(x, vectorizer))


In [43]:
books["user_id"] = "-1"

/var/folders/xz/9z84c__j28g8tg28bmcthjj00000gn/T/ipykernel_34151/4141060674.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books["user_id"] = "-1"


In [44]:
books = books[["user_id", "book_id", "rating", "title"]]

In [45]:
books = books[~pd.isnull(books["rating"])]

In [46]:
books.to_csv("my_books.csv")